# SCRIPT 2
Edit raw data and compute velocity components.

Steps:
1. ds_raw: Load an individual raw drifter dataset with GPS, temperature and other variables on non-uniform t.
    Create tgrid_hourly from the original time data.
2. ds_qc: Flag + remove bad GPS data
    - Where the GPS has dropped out (= mode),
    - Where it is outside the North Atlantic box (na_lonlim, na_latlim)
    - Where the velocity is unrealistic (larger than 3 m/s)
    - Where the time series of longitude and latitude (compared to a one-dimensional five-point median filter) is more than five standard deviations from the five-point median)
     Compute the u and v velocities on the QC GPS fixes
4. ds_lowess: Smooth the GPS (and velocities) using the Lowess filter*.  *Shane's code online doesn't seem to smooth velo with a Lowess
5. ds_hourly: Hourly interpolate the resultant ds_lowess back onto the tgrid_hourly.  Interpolate the other parameters from ds_raw onto the hourly grid (temperature, etc)
    > 'linear' for continuous variables
    > 'nearest neighbour' for discrete ones

6. save as netcdf file

Last modified: 5 Sep 2021

In [21]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# Import modules

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import datetime
import gsw
import os
import glob
import sys
import re
from scipy import stats
import time






%load_ext autoreload
%autoreload 2

# Local import
# > Make sure SIO_wrap dir is on the same path as this script.

from SIO_wrap import dir_tree, fnames
from SIO_wrap.lowess import LatLonLocalWess
from SIO_wrap import jlab_python as jlab
from SIO_wrap import drifter_qc as dqc

from setdir import *

##################---------   LOCAL FUNCTIONS    ---------##################

def latlon_extremes(var, igood):
    """
    var   : xarray dataset that containg the 'flag' and lat/lon variables.
    igood : int or float; value of the good flag

    Prints the min/max of lat and lon.

    <!> This function breaks if the lat/lon variable names change.
    """
    # Select lat/lon where flag is good
    bool_cond = var.flag.values==igood

    lonmin = var.GPS_Longitude_deg.values[bool_cond].min().astype('str')
    latmin = var.GPS_Latitude_deg.values[bool_cond].min().astype('str')
    lonmax = var.GPS_Longitude_deg.values[bool_cond].max().astype('str')
    latmax = var.GPS_Latitude_deg.values[bool_cond].max().astype('str')
    
    print('Lon in ('+lonmin+', '+lonmax+'), Lat in ('+latmin+', '+latmax+')')
#    print('lat min: ', var.GPS_Latitude_deg.values[bool_cond].min())
#    print('lat max: ', var.GPS_Latitude_deg.values[bool_cond].max())
#    print('lon min: ', var.GPS_Longitude_deg.values[bool_cond].min())
#    print('lon max: ', var.GPS_Longitude_deg.values[bool_cond].max())
    


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# User edits

In [5]:
# Path for the output data
data_dir = dir_tree.dir_out

#--------------
# Flag value - good=1, bad=4
good_flagval = 1
bad_gps_flagval = 4 
bad_vel_flagval = 3

# FLAG 1: Latitude values less than this threshold are flagged + removed?
latbad_threshold = 0

# FLAG 2: North Atlantic box; values outside this box are flagged + removed?
# Defined from -180 to 180 (lon) and -90 to 90 (lat)
na_lonlim = [-80, 30]
na_latlim = [40, 80]

#--------------
# Speed threshold; values outside [-3, 3] m/s are flagged + removed?
vel_threshold = 3 # in m/s

#--------------
# LOWESS params
poly_order = 1
bandwidth = 2

#------------
# Names of variables
latname = 'GPS_Latitude_deg'
lonname = 'GPS_Longitude_deg'
uvelname = 'uvel'
vvelname = 'vvel'

#--------------
# Time formats
tstamp_strftime = '%Y%m%d'  # Filename timestamp
timcol_strftime = '%Y-%m-%d %H:%M:%S'  # Convert text to datetime format 

# Reference date for computing time in seconds
# Can use an earlier time reference if data start before 2000
ref_time = datetime.datetime(2000, 1, 1)

#--------------
# List of variable names split between float/int types based on whether the 
# variables are continuous or discrete, respectively.
# <!> If the names of variables change, update the lists by printing a list of 
# all the names from the raw datafile: list(xarrayDataset.keys())

integ_vars = ['Drogue_cnts', 'GPS_HDOP', 'GPS_FixDelay', 'GPS_TTFF', 
              'GPS_NumSat', 'SBD_Transmit_Delay', 'SBD_Retries']

float_vars = [lonname, latname, 'SST_degC',
              'SLP_mB', 'Battery_volts', uvelname, vvelname]


# Step 1. Load raw data

Check the list of PIDs and decide which ones need to be updated.

In [28]:
# Check the list of Platform IDs for TERIFIC
PID = pd.read_csv(cat_proc_path('PID_list.txt'), header='infer', index_col=0)


# Check whether a given Platform ID has been processed and is up-to-date
# If not, then add it to the update list
PID_to_update = []
counter = 0


# Extract a list with the names of existing raw data files.
for i in range(len(PID)):
    pid1 = (PID["PID"].values)[i].astype('str')
    PID1 = (PID["PID"].values)[i]

    # Extract a list with the names of existing raw data files.
    fname = 'pid'+str(PID1)+'_*'
    existing_files = glob.glob(cat_interim_path(fname))

    # Check whether any files exist in the intermediate directory for that PID
    if len(existing_files) > 0:

        # Extract the end date from the filename
        existing_files = sorted(existing_files)
        # Needs existing_files to be sorted() so the last one is latest
        end_date = (existing_files[-1])[-11:-3] 
        t1 = datetime.datetime.strptime(end_date, '%Y%m%d')  

        # Check what the latest raw file was
        raw_files = glob.glob(cat_raw_path(fname))
        if not len(raw_files) > 0:
            print('No raw file for that PID!')

        # Check the end date
        raw_files = sorted(raw_files)
        end_date_raw = (raw_files[-1])[-11:-3]
        t_raw = datetime.datetime.strptime(end_date_raw, '%Y%m%d')

        if t1==t_raw:
            print('pid('+pid1+') - Up to date processed file exists')

        else:
            counter += 1
            PID_to_update.append(PID1)
            print(str(counter)+'. pid('+pid1+') - Ended:'+end_date
                  +', New end:'+t_raw)

    else:
        counter += 1
        print(str(counter)+'. pid('+pid1
              +') - No previous raw data files.')
        PID_to_update.append(PID1)

pid(300234066519040) - Up to date processed file exists
pid(300234068243970) - Up to date processed file exists
1. pid(300234068243460) - No previous raw data files.
pid(300234068342280) - Up to date processed file exists
pid(300234066519050) - Up to date processed file exists
2. pid(300234068343310) - No previous raw data files.
pid(300234068342290) - Up to date processed file exists
3. pid(300234068343830) - No previous raw data files.
4. pid(300234068242460) - No previous raw data files.
pid(300234068244510) - Up to date processed file exists
5. pid(300234066516000) - No previous raw data files.
6. pid(300234068343840) - No previous raw data files.
pid(300234066518050) - Up to date processed file exists
7. pid(300234068342820) - No previous raw data files.
8. pid(300234066517030) - No previous raw data files.
9. pid(300234068242470) - No previous raw data files.
10. pid(300234066516010) - No previous raw data files.
11. pid(300234068242990) - No previous raw data files.
12. pid(3002

In [29]:
# Load a raw file to process it
start_time = time.time()

for i in range(len(PID_to_update)):
    pid1 = (PID_to_update[i]).astype('str')
    PID1 = PID_to_update[i]

    ########################################################################
    ######## Load the raw data
    fname = 'pid'+str(PID1)+'_*'
    raw_files = glob.glob(cat_raw_path(fname))
    raw_files = sorted(raw_files)
    # Find the most recent file (alphabetically, rather than by time)
    raw_files = raw_files[-1]
    # Open the dataset
    ds_raw = xr.open_dataset(raw_files)
    # Some didn't load as sorted by time.
    ds_raw = ds_raw.sortby('time', ascending=True)
    print(fname+' - processing')
    
    #--------------------------------------
    # Total number of points 
    total_points = int(ds_raw.time.size)

    # Compute a time vector in seconds
    dtime_sec = (pd.to_datetime(ds_raw.time.values) - 
                 ref_time).total_seconds()
    ds_raw['time_seconds'] = ("time", dtime_sec)
    # Create the hourly time grid
    tgrid_hourly, tgrid_sec = dqc.create_hourly_grid(ds_raw["time"], ref_time)

    # Find the modes for lat and lon
    mode1 = stats.mode(ds_raw[latname])
    lat_mode = mode1.mode
    mode_count = mode1.count
    mode1 = stats.mode(ds_raw[lonname])
    lon_mode = mode1.mode

    
    # Create a dictionary of attributes
    qc_attr_dict = {"total_points_orig": total_points}

    ########################################################################
    ######## Step 2. Create ds_qc the quality-controlled GPS fixes
    """

    ds_qc: Flag + remove bad GPS data
    - Where the GPS has dropped out (= mode),
    - Where it is outside the North Atlantic box (na_lonlim, na_latlim)
    - Where the velocity is unrealistic (larger than 3 m/s)
    - Where the time series of longitude and latitude (compared to a one-dimensional five-point median filter) is more than five standard deviations from the five-point median)

    Compute the u and v velocities on the QC GPS fixes
    """

    ds_qc = ds_raw

    # Remove variables we won't use here
    fields_to_remove = ['Battery_volts', 'GPS_HDOP', 'GPS_FixDelay',
                        'GPS_TTFF', 'GPS_NumSat', 'SBD_Transmit_Delay',
                        'SBD_Retries', 'SST_degC', 'SLP_mB']

    ds_qc = ds_qc.drop(fields_to_remove, errors='ignore')

    # Add a field FLAG with the same time coordinate.
    # Start by labelling all data 'good' (flag=igood)
    ds_qc["flag"] = ('time', good_flagval * np.ones(ds_qc.time.shape, 
                                                     dtype=np.int8))


    # Where the GPS has dropped out (= mode),
    # Based on previous investigations, there are a lot of -90 values for 
    # GPS_Latitude_deg.  
    if mode_count > 100:
        # Kind of arbitrary but only do this if there are a lot of mode value
        ds_qc = dqc.flag_gps_dropout(ds_qc, latname, 
                                  lat_mode, bad_gps_flagval)

        ds_qc = dqc.flag_gps_dropout(ds_qc, lonname, 
                                  lon_mode, bad_gps_flagval)

    latlon_extremes(ds_qc, good_flagval)

    # Print update on flagged data
    # Need to save this to the output variable instead.
    # Create a dictionary of attributes
#    print("\n> > Flags - stage 1 < <\n")
    num_flags1 = dqc.num_ibad(ds_qc, bad_gps_flagval)
    num_percent1 = num_flags1 / total_points
    percentage = "{:.1%}".format(num_percent1)
#    print("Flagged data as percentage: %s\n" % percentage)
    qc_attr_dict["GPS_dropouts"] = num_flags1

    # Based on the region
    ds_qc = dqc.flag_gps_region(ds_qc, latname,
                              na_latlim, bad_gps_flagval)
    ds_qc = dqc.flag_gps_region(ds_qc, lonname,
                             na_lonlim, bad_gps_flagval)

    latlon_extremes(ds_qc, good_flagval)


    # Print update on flagged data
#    print("\n> > Flags - stage 1 + 2 < <\n")
    num_flags2 = dqc.num_ibad(ds_qc, bad_gps_flagval)
    num_percent2 = num_flags2 / total_points
    percentage = "{:.1%}".format(num_percent2)
#    print("Total flagged data as percentage: %s\n" % percentage)

    num_flags2_diff = num_flags2 - num_flags1
#    print("Flagged data at this step only: %s\n" % str(num_flags2_diff))
    qc_attr_dict["GPS_out_of_region"] = num_flags2_diff



    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    # Remove flagged data, otherwise it might affect the lowess filtering

#    print("Removing flagged data. \n")
    ds_qc = ds_qc.where(ds_qc.flag==good_flagval, drop=True) 

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
    # velocity calculation
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
    GPSlat = ds_qc[latname].to_numpy()
    GPSlon = ds_qc[lonname].to_numpy()
    u_orig, v_orig = jlab.latlon2uv(ds_qc['time_seconds'], GPSlat, GPSlon)
    velmag = np.sqrt(np.square(u_orig) + np.square(v_orig))

    # Plot to verify
    if 0:
        plt.plot(velmag)    

    ds_qc[uvelname] = ('time', u_orig)
    ds_qc[vvelname] = ('time', v_orig)

    # Flag the bad stuff
    ds_qc.flag.values[velmag > vel_threshold] = bad_vel_flagval
    num_flags3 = dqc.num_ibad(ds_qc, bad_vel_flagval)
    qc_attr_dict["velocity_exceed_threshold"] = num_flags3
    
    # Need to count how many were removed here

    # Removing flagged data
    ds_qc = ds_qc.where(ds_qc.flag==good_flagval, drop=True) 

    if 0:
        GPSlat = ds_qc[latname].to_numpy()
        GPSlon = ds_qc[lonname].to_numpy()

        u_orig, v_orig = jlab.latlon2uv(ds_qc['time_seconds'], GPSlat, GPSlon)
        velmag = np.sqrt(np.square(u_orig) + np.square(v_orig))
        plt.plot(velmag)


    print('Flagged '+str(num_flags1)+' for GPS dropouts, '+str(num_flags2_diff)+' for region, '+str(num_flags3)+' for velo')



    ########################################################################
    ######## Step 3. Run Lowess filter

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
    # Create entries for lowess lat/lon
    time1 = ds_qc.time.values
    ds_lowess = xr.Dataset(coords=dict(time=(["time"], time1)))
    ds_lowess = ds_lowess.assign_attrs(ds_qc.attrs)

    ds_lowess[latname] = ('time', np.ones(len(ds_qc.time)))
    ds_lowess[lonname] = ('time', np.ones(len(ds_qc.time)))


    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    # LOWESS: Locally Weighted Scatterplot Smoother (Elipot et al 2016)
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    time_sec = ds_qc['time_seconds'].values
    lon = ds_qc[lonname].values
    lat = ds_qc[latname].values
    llat, llon = LatLonLocalWess(time_sec, lon, lat, poly_order, bandwidth)

    ds_lowess[latname] = ('time', llat)
    ds_lowess[lonname] = ('time', llon)

    # Cannot use the same code to Lowess filter the velocities since it treats 
    # latitude and longitude differently.
    ds_lowess


    ########################################################################
    ######## Step 4. Interpolate hourly

    # Revised from Oana's method (using arrays) to use dataset in xarray.


    # Empty lists for storing the hourly data and new column names
    hourly_arr = []
    arr_names = []


    total_points_qc = int(ds_raw.time.size)
    qc_attr_dict["qc_points"] = total_points_qc


    # Create the new hourly xarray dataset
    ds_hourly = xr.Dataset(coords=dict(time=(["time"], tgrid_hourly)))
    ds_hourly = ds_hourly.assign_attrs(ds_lowess.attrs)
    ds_hourly
    time_sec = ds_qc.time_seconds
    time_sec_raw = ds_raw.time_seconds

    # Get the variable names
    varnames_list = list(ds_raw.keys())
    varnames_qc_list = list(ds_qc.keys())
    varnames_lowess_list = list(ds_lowess.keys())

    # Create temporary array to store hourly interp data
    c_len = len(tgrid_hourly)
    arr = np.ones((c_len,))

    # Iterate through the variable names and set the interpolation method 
    # based on the dtype of the data
    k = 0
    for varname in set(varnames_list + varnames_qc_list):
        if (varname in varnames_lowess_list):
            # Lowess filtered variables (GPS lat and lon)
            if varname in integ_vars:
                interp_method = 'nearest'
            elif varname in float_vars:
                interp_method = 'linear'

            var = ds_lowess[varname].values
            fc_interp = interp1d(time_sec, var, interp_method,
                                 fill_value='extrapolate')
            arr = fc_interp(tgrid_sec)

            ds_hourly[varname] = ('time', arr)
            #            print('Using lowess for '+varname)

        elif (varname in varnames_qc_list):
            # QC drifter variables (velocity)
            if varname in integ_vars:
                interp_method = 'nearest'
            elif varname in float_vars:
                interp_method = 'linear'

            var = ds_qc[varname].values
            fc_interp = interp1d(time_sec, var, interp_method,
                                 fill_value='extrapolate')
            arr = fc_interp(tgrid_sec)

            ds_hourly[varname] = ('time', arr)
             #           print('Using qc for '+varname)


        elif (varname in integ_vars) or (varname in float_vars):
            # Raw drifter variables
            if varname in integ_vars:
                interp_method = 'nearest'
            elif varname in float_vars:
                interp_method = 'linear'

            var = ds_raw[varname].values
            fc_interp = interp1d(time_sec_raw, var, interp_method,
                                 fill_value='extrapolate')
            arr = fc_interp(tgrid_sec)

            ds_hourly[varname] = ('time', arr)
        else:
            print("Variables not interpolated hourly: < %s >\n" % varname)


    # attributes for Dataset
    #ds_hourly.time_seconds.attrs["units"] = "seconds since %s" % str(ref_time.strftime(timcol_strftime))
    ds_hourly[lonname].attrs["long_name"] = "longitude_lowess"
    ds_hourly[latname].attrs["long_name"] = "latitude_lowess"
    ds_hourly[uvelname].attrs["units"] = "m/s"
    ds_hourly[vvelname].attrs["units"] = "m/s"
    ds_hourly[uvelname].attrs["long_name"] = "zonal_velocity"
    ds_hourly[vvelname].attrs["long_name"] = "meridional_velocity"




    # Update attributes
    dstr = datetime.datetime.today()
    dstr = dstr.replace(hour=0, minute=0, second=0, 
                        microsecond=0).strftime('%Y-%m-%d')

    maxtime = tgrid_hourly.max()
    maxtimestr = pd.to_datetime(maxtime).strftime('%Y-%m-%dT%H:%M:%S')

    procstr = f'Flagged GPS out of LabSea, ran Lowess filter with poly_order({poly_order}) and bandwidth({bandwidth})'

    # Create a dictionary of attributes
    attr_dict = {"Platform_ID": PID1,
                 "End Time": maxtimestr,
                 "Date created": dstr,
                 "Processing": procstr,
                }


    ds_hourly = ds_hourly.assign_attrs(attr_dict)
    ds_hourly = ds_hourly.assign_attrs(qc_attr_dict)

    # Output filename (will be the same as raw, but in 02-intermediate/)
    data_fname = raw_files[-30:-3]+'.nc'
    data_fpath = cat_interim_path(data_fname)

    ds_hourly.to_netcdf(data_fpath)

    ds_hourly
    current_time = time.time()
    elapsed_time = current_time-start_time
    print('Done in '+str(int(elapsed_time))+' seconds')
    
    PID_to_update.remove(PID1)


TypeError: list indices must be integers or slices, not str

300234068243460

In [ ]:
if 0:

    # ~ ~ print update ~ ~ 
    if len(existing_files) > 0:
        print("Existing raw data files: \n%s\n" % existing_files)
    else:
        sys.exit("No previous raw data files.\n")

    # ~ ~ filenaming convention ~ ~
    # If there are multiple files with raw data (i.e. non-updated datasets), select 
    # the latest one updated.
    # The file names are distinguished by the timestamp appended to the filename 
    # and has <tstamp_strftime> format (see 'user edits' section).
    # The data are cropped such that the last day is fully sampled (spans 0h-23h).
    # The timestamp in the filename is the latest downloaded fully sampled day.

    # Extract the timestamp part of the filename(s) in a list
    tstamp = [date for file in existing_files 
                for date in re.findall("(\d{8})", file)]

    # Convert to datetime and pick the most recent timestamp
    tstamp_date = pd.to_datetime(tstamp, format=tstamp_strftime)
    fname_timestamp = tstamp[tstamp_date.argmax()]

    # Load the raw file with the latest timestamp
    ds_fname = f"{fnames.fname_rawdata}{fname_timestamp}.nc"
    ds_fpath = os.path.join(data_dir, ds_fname)

    print("Opening file: %s\n" % ds_fpath)
    ds_raw = xr.open_dataset(ds_fpath)
